In [40]:
from bs4 import BeautifulSoup as bsoup
import pandas as pd 
import numpy as np
import requests
import pprint


In [41]:
linkHouses = 'https://www.buyrentkenya.com/houses-for-rent'

In [42]:
# doc =requests.get('https://www.buyrentkenya.com/houses-for-rent?page=11')
# soup = bsoup(doc.content, 'html.parser')

# max_len= soup.find_all('div',class_='flex items-center justify-center text-xl font-bold leading-7 text-grey-900')
# for i in max_len:
#     print(i.text)
    
# max_len = np.int64(max_len[-1].string.strip())


In [43]:
links =[]

for i in range(2,51):
    link = (linkHouses + f"?page={i}")
    links.append(link)


In [44]:
main_soup =[]
for i in links[1:]:
    text = requests.get(i)
    soup = bsoup(text.text,'lxml')
    main_soup.append(soup)

len(main_soup)

48

In [45]:
links

['https://www.buyrentkenya.com/houses-for-rent?page=2',
 'https://www.buyrentkenya.com/houses-for-rent?page=3',
 'https://www.buyrentkenya.com/houses-for-rent?page=4',
 'https://www.buyrentkenya.com/houses-for-rent?page=5',
 'https://www.buyrentkenya.com/houses-for-rent?page=6',
 'https://www.buyrentkenya.com/houses-for-rent?page=7',
 'https://www.buyrentkenya.com/houses-for-rent?page=8',
 'https://www.buyrentkenya.com/houses-for-rent?page=9',
 'https://www.buyrentkenya.com/houses-for-rent?page=10',
 'https://www.buyrentkenya.com/houses-for-rent?page=11',
 'https://www.buyrentkenya.com/houses-for-rent?page=12',
 'https://www.buyrentkenya.com/houses-for-rent?page=13',
 'https://www.buyrentkenya.com/houses-for-rent?page=14',
 'https://www.buyrentkenya.com/houses-for-rent?page=15',
 'https://www.buyrentkenya.com/houses-for-rent?page=16',
 'https://www.buyrentkenya.com/houses-for-rent?page=17',
 'https://www.buyrentkenya.com/houses-for-rent?page=18',
 'https://www.buyrentkenya.com/houses-f

In [46]:
spans =[]
for i in main_soup:
   spans.append( i.find_all('span',class_='relative top-[2px] hidden md:inline'))
    
spans

houses=[]
for span in spans:
   houses.append([str(i.text).strip() for i in span])
   

# for i in houses:
#    print(len(i))

# houses = [i for j in houses for i in j]
len(houses)


48

In [47]:
divs = []
for i in main_soup:
    div =i.find_all('div',class_='flex items-center justify-center text-xl font-bold leading-7 text-grey-900')
    divs.append(div)  

        

prices=[]
for div in divs:
    text = [str(i.text).strip().split()[1] for i in div[::2]]
    prices.append(text)
    

# for i in prices:
#    print(len(i))
 
 
# prices      
    

len(prices)
 
# prices.append(text.split()[1])   


48

In [48]:
mainHouses = [i for i,j in zip(houses,prices) if len(i)==len(j)]
mainPrices = [i for i,j in zip(prices,houses) if len(i)==len(j)]

In [49]:
mainPrices =[i for j in mainPrices for i in j]
mainHouses = [i for j in mainHouses for i in j]

In [50]:
dictHouses = {'Houses':mainHouses, 'Prices':mainPrices}
dictHouses

{'Houses': ['1 Bed House with Garden at Glory Drive',
  '4 Bed Villa with En Suite at Grevillea Grove Road',
  '4 Bed Villa with En Suite in Runda',
  '4 Bed House with En Suite in Runda',
  '4 Bed House with En Suite in Runda',
  '4 Bed House with En Suite in Kiambu Road',
  '4 Bed Townhouse with En Suite in Spring Valley',
  '4 Bed House with En Suite in Runda',
  '4 Bed House with En Suite at Chalbi Drive',
  '4 Bed Townhouse with En Suite at Kinanda Rd',
  '4 Bed Townhouse with En Suite at Mushroom Gardens',
  '4 Bed House with En Suite at Rosslyn',
  '4 Bed Townhouse with En Suite in Runda',
  '4 Bed House with En Suite in Runda',
  '5 Bed House with En Suite at Runda',
  '4 Bed House with En Suite at Kiambu Road',
  '6 Bed House with En Suite in Runda',
  '4 Bed House with En Suite in Runda',
  '5 Bed Villa with En Suite at Tala Rd',
  '5 Bed Townhouse with En Suite in Lavington',
  '4 Bed Townhouse with En Suite at Lavington',
  '6 Bed House with En Suite in Nyari',
  '4 Bed Tow

In [51]:
dfHouses = pd.DataFrame.from_dict(data=dictHouses)
dfHouses

,Houses,Prices
0,1 Bed House with Garden at Glory Drive,"90,000"
1,4 Bed Villa with En Suite at Grevillea Grove Road,"300,000"
2,4 Bed Villa with En Suite in Runda,"500,000"
3,4 Bed House with En Suite in Runda,"280,000"
4,4 Bed House with En Suite in Runda,"450,000"
...,...,...
694,4 Bed House with En Suite in Spring Valley,"280,000"
695,3 Bed House with En Suite in Hill View,"320,000"
696,4 Bed House with En Suite in Kitisuru,"320,000"
697,4 Bed House with En Suite in Muthaiga,"450,000"


In [52]:
# dfHouses.to_excel('houses.xlsx',index=False)